In [1]:
! pip install albumentations==0.4.6

In [2]:
!pip install torch torchvision torchaudio

In [3]:
from time import time
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import cv2

from sklearn.model_selection import train_test_split, GroupKFold, StratifiedKFold

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.ops import nms

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

torch.cuda.empty_cache() 

import os

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
!cd gdrive/MyDrive/TipsAndTripcksProject\ / 

In [6]:
os.chdir('gdrive/MyDrive/TipsAndTripcksProject /')

In [7]:
from data_preparation import LISADataset, collate_fn, getTrainTransform, data_loader, video_predict

In [8]:
###

### Data preparation

In [9]:
from google.colab.patches import cv2_imshow

In [10]:
dataset = LISADataset()
train, test = data_loader(batch_size = 1, transform=getTrainTransform())

In [11]:
t = next(iter(train))

In [12]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
N_CLASS = 4 

In [13]:
INP_FEATURES = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(INP_FEATURES, N_CLASS)

In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [15]:
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(params)

lr_scheduler  = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [16]:
EPOCHS = 4

In [17]:
class LossAverager:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
      

In [18]:
lossHist = LossAverager()
valLossHist = LossAverager()

In [ ]:
for epoch in range(EPOCHS):
    
  
    model.train()
    lossHist.reset()
    
    for images, targets in tqdm(train):
        
        images = torch.stack(images).to(device)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        bs = images.shape[0]
        
        loss_dict = model(images, targets)
        
        totalLoss = sum(loss for loss in loss_dict.values())
        lossValue = totalLoss.item()
        
        lossHist.update(lossValue,bs)

        optimizer.zero_grad()
        totalLoss.backward()
        optimizer.step()
    
    if lr_scheduler is not None:
        lr_scheduler.step(totalLoss)

    # print(f"[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}]")
    print(f"Epoch {epoch}/{EPOCHS}")
    print(f"Train loss: {lossHist.avg}")
    
    torch.save(model.state_dict(), 'Models/fasterrcnn_resnet_v3{}_fpn.pth'.format(epoch))


 26%|██▌       | 2652/10220 [23:18<1:27:41,  1.44it/s]

In [ ]:
!ls Models/